In [1]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/csc-578-final-project-fall-2023/lsst-train.csv
/kaggle/input/csc-578-final-project-fall-2023/lsst-test.csv


First read in the training set as you normally do/did.  My suggestion is to read it into a pandas dataframe.

In [2]:
parent_train_ = pd.read_csv("/kaggle/input/csc-578-final-project-fall-2023/lsst-train.csv")
parent_test = pd.read_csv("/kaggle/input/csc-578-final-project-fall-2023/lsst-test.csv")
parent_train = parent_train_

You must convert the target column ('target'; y) to integers.

- The integers should be the 0-based indices of the target classes.  For example, 'c-15' --> 0, 'c-16' --> 1, and so on.
- It depends on how you write the code, but you can separate the 'target' y column from the x part (216 features).  If you do, you may want to reshape the new integer-target column (to (-1, 1)) to ensure a numpy column vector/array.
- Note he original nominal/string target column won't be needed after integer conversion -- you only use the converted column

In [3]:
parent_test.head()

,f-1-1,f-1-2,f-1-3,f-1-4,f-1-5,f-1-6,f-1-7,f-1-8,f-1-9,f-1-10,...,f-6-27,f-6-28,f-6-29,f-6-30,f-6-31,f-6-32,f-6-33,f-6-34,f-6-35,f-6-36
0,25.575000,25.5650,6.8052,-26.012,-55.1290,-62.0750,-43.3400,-14.26500,4.03600,2.8128,...,114.9600,116.3700,69.0540,11.8250,29.5960,55.14500,1.0701,-31.4420,20.914,39.9590
1,2.432400,10.4920,16.6900,14.609,3.8112,-8.9224,-15.1370,-11.82400,-3.60240,2.0792,...,16.7950,3.3646,5.7902,52.7540,-19.5210,-9.35210,63.4920,-17.8800,-40.938,6.2865
2,-0.166710,-1.0756,-0.8531,1.626,-1.8959,2.4717,-0.2629,-1.43750,0.89766,-2.4570,...,-3.3971,-8.2973,8.8071,-3.6586,-6.4716,0.87079,2.6530,2.2138,-1.323,-2.1806
3,-4.800300,-3.5558,-5.2497,-10.011,-10.9800,-3.5785,4.9622,2.28200,-11.07200,-17.1180,...,2.9662,-6.1161,-34.6170,-96.5080,-94.9050,-8.84960,48.7190,-15.2810,-15.786,34.8360
4,-0.048579,-11.9320,-25.6890,-35.259,-36.0600,-27.6460,-13.9240,-0.79703,7.31190,9.8219,...,63.1690,64.7760,53.0120,-83.1160,-50.6480,79.74300,-16.5700,74.2040,72.036,100.2200


In [4]:
y = parent_train['target'].values
parent_train = parent_train.drop('target', axis=1)
parent_train.head()

,f-1-1,f-1-2,f-1-3,f-1-4,f-1-5,f-1-6,f-1-7,f-1-8,f-1-9,f-1-10,...,f-6-27,f-6-28,f-6-29,f-6-30,f-6-31,f-6-32,f-6-33,f-6-34,f-6-35,f-6-36
0,2.57770,-1.18830,-4.98950,-6.4301,-4.6503,-1.02450,1.90420,2.48850,1.360500,0.90286,...,13.5590,22.36600,-24.7480,-34.21500,7.7464,-5.8332,-0.71757,8.16160,-29.6940,-27.5700
1,6.67580,22.54300,44.02700,66.9610,86.0820,96.80400,96.76000,86.49400,69.094000,48.97100,...,-22.6270,19.26700,28.5110,0.97578,-28.0270,-18.0720,-4.00020,-43.07500,-4.7283,-43.0070
2,0.37554,-3.10860,-0.26009,3.2281,-1.9951,0.12797,0.25844,-0.25394,-0.091828,2.41440,...,2.3504,-4.02610,14.5270,50.00500,-13.8860,3.3589,5.00450,-0.47256,-2.7713,1.9379
3,0.51852,0.12078,1.53240,5.5783,11.5340,17.01300,18.97100,15.38700,6.685500,-4.07800,...,17.4670,23.24700,31.9850,26.62100,-12.2560,3.4805,23.80500,-32.36400,-15.1610,-14.4500
4,1.47150,1.44510,2.91330,1.0015,-1.2503,1.55160,1.73170,0.47796,0.539860,-1.71690,...,2.1311,-0.18477,3.8533,-5.80000,1.4909,2.7581,-3.16650,-0.30058,6.6334,7.3427


In [5]:
classes = np.unique(y) 
class_to_idx = {c: i for i, c in enumerate(classes)}
y_int = np.array([class_to_idx[c] for c in y]) 
y_int = y_int.reshape(-1,1)
print(y_int.shape)

(3356, 1)


In [6]:
classes_ = np.unique(y) 
print(len(classes_))
class_to_idx = {c: i for i, c in enumerate(classes_)}
y_int = np.array([class_to_idx[c] for c in y]) 
y_int = y_int.reshape(-1,1)
print(y_int)

11
[[8]
 [0]
 [2]
 ...
 [4]
 [2]
 [8]]


In [7]:
timesteps = 6
features_per_timestep = 36
X_train = parent_train.values.reshape(-1, timesteps, features_per_timestep)
input_shape = X_train.shape[1:]
print(X_train.shape)

(3356, 6, 36)


In [8]:
timesteps = 6
features_per_timestep = 36
Ytest = parent_test.values.reshape(-1, timesteps, features_per_timestep)
print(Ytest.shape)

(1439, 6, 36)


In [9]:
from sklearn.model_selection import train_test_split 
X_train_, X_val, y_train_, y_val = train_test_split( X_train, y_int, test_size=0.2, random_state=42) 

## Error Analysis for the model to check the Accuracy

In [10]:
from sklearn.metrics import precision_score, recall_score, fbeta_score
def error_analysis(model, X_val, y_val):
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    # Calculate Precision, Recall, and F2-score
    precision = precision_score(y_val, y_pred_classes, average='weighted')
    recall = recall_score(y_val, y_pred_classes, average='weighted')
    f2_score = fbeta_score(y_val, y_pred_classes, beta=2, average='weighted')

    # Print error analysis metrics
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F2 Score: {f2_score:.4f}")


In [11]:
import keras
import tensorflow as tf
from keras.regularizers import l1, l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

classes = len(classes_) 
def model2():
    input_shape = X_train.shape[1:] 
    normalizer = tf.keras.layers.Normalization() 
    normalizer.adapt(X_train)
    inputs = tf.keras.Input(shape=input_shape)
    x = normalizer(inputs)
    #x = tf.keras.layers.LSTM(8)(x)
    #x = tf.keras.layers.GRU(8,activation="relu", dropout=0.5,)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x  = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(64, activation='softmax')(x)
    outputs = tf.keras.layers.Dense(classes, activation='softmax')(x) 
    lstm_model = keras.Model(inputs, outputs)
    lstm_model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy', 
                   metrics=['accuracy'])
    lstm_model.summary()
    return lstm_model 

def model3():
    input_shape = X_train.shape[1:] 
    normalizer = tf.keras.layers.Normalization() 
    normalizer.adapt(X_train)
    inputs = tf.keras.Input(shape=input_shape)
    x = normalizer(inputs)
    #x = tf.keras.layers.LSTM(8)(x)
    #x = tf.keras.layers.GRU(8,activation="relu", dropout=0.5,)(x)
    x = tf.keras.layers.SimpleRNN(
    32,
    activation="tanh",
    use_bias=True,
    kernel_initializer="glorot_uniform",
    recurrent_initializer="orthogonal",
    bias_initializer="zeros",
    kernel_regularizer=l1(0.01),
    recurrent_regularizer=None,
    bias_regularizer=None,
    )(x)
    x = tf.keras.layers.Flatten()(x)
    """
    x = tf.keras.layers.Dense(64, activation='softmax')(x)
    x  = tf.keras.layers.Dropout(0.2)(x)"""
    outputs = tf.keras.layers.Dense(classes, activation='softmax')(x) 
    lstm_model = keras.Model(inputs, outputs)
    # Define a custom learning rate
    custom_lr = 0.001

    # Create an Adam optimizer with the specified learning rate
    optimizer = Adam(learning_rate=custom_lr)
    lstm_model.compile(optimizer=optimizer,
                   loss='sparse_categorical_crossentropy', 
                   metrics=['accuracy'])
    lstm_model.summary()
    return lstm_model 
def model4():
    from tensorflow.keras.models import Model
    from tensorflow.keras import layers
    # Assuming 'X_train' and 'num_classes' are defined elsewhere
    input_shape = X_train.shape[1:] 
    normalizer = tf.keras.layers.Normalization() 
    normalizer.adapt(X_train)
    inputs = tf.keras.Input(shape=input_shape)
    x = normalizer(inputs)
    x = layers.Conv1D(filters=64, kernel_size=5, activation='relu',kernel_regularizer=l1(0.01))(x)
    out = layers.Dense(classes, activation='softmax')(x)
    # Define the model
    model = Model(inputs, out)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],callbacks=[early_stopping]
    )

    return model
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, Dense, Dropout
import tensorflow as tf
from tensorflow.keras.layers import LSTM, BatchNormalization, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
def model_with_dropout():####as of now this is the best model.
    input_shape = X_train.shape[1:] 
    normalizer = tf.keras.layers.Normalization() 
    normalizer.adapt(X_train)
    inputs = tf.keras.Input(shape=input_shape)
    x = normalizer(inputs)
    x = LSTM(128, activation='relu', dropout=0.3, recurrent_dropout=0.3)(x)
    x = BatchNormalization()(x)
    # Additional Dense layers (you can uncomment and adjust as needed)
    """
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    """
    # Dense layers for classification with dropout
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.2)(x)
    out = Dense(classes, activation='softmax')(x)
    # Define the model
    model = Model(inputs, out)
    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Define the optimizer with learning_rate
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
def multi_model():####as of now this is the best model.
    input_shape = X_train.shape[1:] 
    normalizer = tf.keras.layers.Normalization() 
    normalizer.adapt(X_train)
    inputs = tf.keras.Input(shape=input_shape)
    x = normalizer(inputs)
    rnn = layers.Bidirectional(layers.SimpleRNN(64, return_sequences=True, kernel_regularizer=l1(0.01)))(inputs)  

    # 1D Conv Layer 
    conv = layers.Conv1D(filters=32, kernel_size=3, activation='relu',kernel_regularizer=l2(0.01))(rnn)

    # ConvLSTM Layer
    #conv_lstm = layers.ConvLSTM2D(filters=64, kernel_size=(1, 3))(conv) 

    # MLP Classifier  
    x = layers.GlobalAveragePooling1D()(conv)
    x = layers.Dropout(0.5)(x)  
    outputs = layers.Dense(classes, activation='softmax')(x)
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.2)(x)
    out = Dense(classes, activation='softmax')(x)
    # Define the model
    model = Model(inputs, out)
    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Define the optimizer with learning_rate
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [12]:
model = model3()
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(X_train_, y_train_, epochs=120, batch_size=32, validation_data=(X_val, y_val),callbacks=[early_stopping])

2023-11-21 23:48:07.966066: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-21 23:48:07.966188: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-21 23:48:07.966279: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-21 23:48:07.966353: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-21 23:48:07.966421: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-21 23:48:07.966650: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-21 23:48:07.966770: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6, 36)]           0         
                                                                 
 normalization (Normalizati  (None, 6, 36)             73        
 on)                                                             
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                2208      
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 11)                363       
                                                                 
Total params: 2644 (10.33 KB)
Trainable params: 2571 (10.04 KB)
Non-trainable params: 73 (296.00 Byte)
________________________

In [13]:
#Error Analysis
error_analysis(model, X_val, y_val)

21/21 [==============================] - 0s 2ms/step
Precision: 0.4337
Recall: 0.4851
F2 Score: 0.4493


/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
#Predicting the model 
preds = model.predict(Ytest,batch_size=100)

15/15 [==============================] - 0s 2ms/step


In [15]:
preds = model.predict(Ytest,batch_size=32)
predictions_df = pd.DataFrame(preds, columns=['c-15', 'c-16', 'c-42', 'c-52', 'c-62', 'c-65', 'c-67', 'c-88', 'c-90', 'c-92', 'c-95'])
predictions_df.index = np.arange(1, len(predictions_df) + 1)
predictions_df['ID'] = predictions_df.reset_index().index
predictions_df.rename_axis('ID', inplace=True)
id_col = predictions_df['ID']  # Extract the 'ID' column
predictions_df.drop(columns=['ID'], inplace=True)  # Remove 'ID' column from DataFrame
predictions_df.insert(0, 'ID', id_col)  # Insert 'ID' column at the beginning
print(predictions_df)
predictions_df.to_csv('submission.csv', index = False,sep=",") 

45/45 [==============================] - 0s 2ms/step
        ID      c-15      c-16      c-42      c-52      c-62      c-65  \
ID                                                                       
1        0  0.007671  0.014225  0.251205  0.063879  0.091716  0.076363   
2        1  0.008125  0.013665  0.224872  0.038708  0.138407  0.065798   
3        2  0.030555  0.011976  0.271053  0.040771  0.063353  0.076106   
4        3  0.005473  0.016339  0.299360  0.047871  0.146319  0.036423   
5        4  0.002148  0.414180  0.000993  0.000449  0.000928  0.151181   
...    ...       ...       ...       ...       ...       ...       ...   
1435  1434  0.019370  0.149233  0.132818  0.011300  0.032539  0.243381   
1436  1435  0.002675  0.736727  0.012920  0.001030  0.005467  0.195414   
1437  1436  0.015176  0.010518  0.196329  0.046280  0.067009  0.087222   
1438  1437  0.013031  0.015026  0.161225  0.030233  0.088651  0.130902   
1439  1438  0.008143  0.005977  0.258396  0.062105  0.11429

In [16]:
df = pd.read_csv("submission.csv")
print(df)

        ID      c-15      c-16      c-42      c-52      c-62      c-65  \
0        0  0.007671  0.014225  0.251205  0.063879  0.091716  0.076363   
1        1  0.008125  0.013665  0.224872  0.038708  0.138407  0.065798   
2        2  0.030555  0.011976  0.271053  0.040771  0.063353  0.076106   
3        3  0.005473  0.016339  0.299360  0.047871  0.146319  0.036423   
4        4  0.002148  0.414180  0.000993  0.000449  0.000928  0.151181   
...    ...       ...       ...       ...       ...       ...       ...   
1434  1434  0.019370  0.149233  0.132818  0.011300  0.032539  0.243381   
1435  1435  0.002675  0.736727  0.012920  0.001030  0.005467  0.195414   
1436  1436  0.015176  0.010518  0.196329  0.046280  0.067009  0.087222   
1437  1437  0.013031  0.015026  0.161225  0.030233  0.088651  0.130902   
1438  1438  0.008143  0.005977  0.258396  0.062105  0.114297  0.043067   

          c-67      c-88      c-90      c-92      c-95  
0     0.051915  0.002751  0.421770  0.000463  0.018043